Trained model download link : [15epoch](https://drive.google.com/file/d/1WNVQy4bvtg4VmDQKLF3-eJfKROltM9TI/view?usp=sharing), [25epoch](https://drive.google.com/file/d/1TVxYdy_XvyzdT3YsjT-gYar123nCU0Hj/view?usp=sharing) model

Pre-trained model download link : [15epoch](https://drive.google.com/file/d/1UDCJ3YllR3RN4QK8NTe5UoTP_s3nbde9/view?usp=sharing)

Run the process in order and change the file paths into your own path. Files will be attached in the github folder.

In [1]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install tensorflow==1.12.0

     |████████████████████████████████| 83.1MB 67kB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 3.1MB 39.6MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [9]:
#!/usr/bin/env python

import argparse
import io
import os
import sys

import tensorflow as tf


def read_image(file):
    """Read an image file and convert it into a 1-D floating point array."""
    file_content = tf.read_file(file)
    image = tf.image.decode_jpeg(file_content, channels=1)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.reshape(image, (1, 64*64))
    return image


def classify(img_file, label_file, graph_file):
    """Classify a character.

    This method will import the saved model from the given graph file, and will
    pass in the given image pixels as input for the classification. The top
    five predictions will be printed.
    """
    labels = io.open(label_file,
                     'r', encoding='utf-8').read().splitlines()

    if not os.path.isfile(img_file):
        print('Error: Image %s not found.' % img_file)
        sys.exit(1)

    # Load graph and parse file.
    with tf.gfile.GFile(graph_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name='hangul-model',
            producer_op_list=None
        )

    # Get relevant nodes.
    x = graph.get_tensor_by_name('hangul-model/input:0')
    y = graph.get_tensor_by_name('hangul-model/output:0')
    keep_prob = graph.get_tensor_by_name('hangul-model/keep_prob:0')

    image = read_image(img_file)
    sess = tf.InteractiveSession()
    image_array = sess.run(image)
    sess.close()
    with tf.Session(graph=graph) as graph_sess:
        predictions = graph_sess.run(y, feed_dict={x: image_array,
                                                   keep_prob: 1.0})
        prediction = predictions[0]

    # Get the indices that would sort the array, then only get the indices that
    # correspond to the top 5 predictions.
    sorted_indices = prediction.argsort()[::-1][:5]
    for index in sorted_indices:
        label = labels[index]
        confidence = prediction[index]
        return label

# Change the path names in classify for your own file
# second arguement should be the label file, third arguement should be the pre-trained model
def tag(image_file):
    return classify(image_file, '/gdrive/My Drive/CS470/teamai/labels/2350-common-hangul.txt', '/gdrive/My Drive/CS470/teamai/save_model_trained/optimized_hangul_tensorflow_2350_25.pb')


In [4]:
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import glob
import pandas as pd

Input image files should be named as the correct Korean character to evaluated the accuracy of the model

In [5]:
#the path in glob should be the path to the image files, also change png into the img files state
images = glob.glob('/gdrive/MyDrive/CS470/teamai/test_letter/all_test_h/*.png')

The unicode type of the parsed image path name is not the same as the model generated result. So the accuracy is now calculated by human labor.

In [10]:
#this will save the images with the tagged unicode into the directory you insert
count = 0
for img in images:
  image = img
  cvimg = cv2.imread(image)
  dst = cv2.bitwise_not(cvimg)
  cv2.imwrite( "checking.jpg", dst )
  newimg = Image.open("checking.jpg")
  resized = newimg.resize((64,64))

  data = np.array(resized)
  red, green, blue = data[:,:,0], data[:,:,1],data[:,:,2]
  for i in range(64):
    for j in range(64):
      imsi = data[i][j]
      if (int(imsi[0])+int(imsi[1])+int(imsi[2])) < 300:
        data[i][j] = [0,0,0]
  newimg1 = Image.fromarray(data)

  newimg1.save('new1.jpeg')
  tagged_letter = tag('/content/new1.jpeg')
  #the path will be the directory you want to save the img files
  name = img.split('/')[-1].split('.')[0]
  print(tagged_letter, name)
  count += 1
print(count)
  

은 은
걔 개
울 울
가 가
에 에
서 서
녀 녀
를 를
보 보
자 자
윤 윤
초 초
시 시
증 증
손 손
딸 딸
슥 수
있 있
었 었
잠 잠
고 고
장 장
지 지
기 기
듯 듯
써 써
철 칠
학 학
아 아
오 오
어 어
제 제
까 까
니 니
둑 둑
모 모
르 르
냥 냥
날 날
쎄 쎄
안 만
물 물
켜 켜
밴 낸
번 번
허 허
탕 탕
재 재
뫼 미
양 양
움 움
킨 킨
처 처
렁 럼
건 건
너 너
사 사
랍 람
킬 길
비 비
려 러
아 다
속 속
엇 엇
하 하
조 조
약 약
돌 돌
리 리
벌 벌
퍽 떡
나 나
꽐 팔
짝 짝
징 징
겅 검
간 간
홱 홱
며 며
바 바
행 행
주 주
읗 음
종 좀
늦 늦
뎨 데
세 세
터 터
걷 걷
록 목
털 덜
마 마
희 희
한 한
찮 참
빤 빤
들 들
여 여
얼 얼
굴 굴
끼 끼
도 도
걸 걸
교 교
것 것
난 난
는 는
늘 늘
네 네
더 더
라 라
런 런
렸 렸
슭 슭
못 못
알 알
앉 앉
을 을
째 째
키 키
쥡 집
본 본
로 로
문 무
킬 킬
이 이
얀 얀
고 그
뛰 뛰
요 요
웨 웨
꾼 꾸
132
